In [1]:
import os
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import timm
from torchvision import transforms

C:\MyPrograms\Anaconda\envs\tf_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# Load EfficientNet B3 without classification head
model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=0)  # Removes FC layer
model.eval()
model.to(device)

# Preprocessing for EfficientNet B3 (as per its training stats)
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
def extract_features_efficientnet(dataset_dir, split):
    mapping_path = os.path.join(dataset_dir, split, f"{split}_mapping.csv")
    df = pd.read_csv(mapping_path, header=None, names=["filename", "class"])

    features = []
    labels = []

    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"EfficientNet {split}"):
        img_path = os.path.join(dataset_dir, split, row["filename"])
        img = Image.open(img_path).convert("RGB")
        img = transform(img).unsqueeze(0).to(device)

        with torch.no_grad():
            feat = model(img).squeeze().cpu().numpy()  # output shape: (1536,)
        
        features.append(feat)
        labels.append(row["class"])

    # Save features and labels
    np.save(os.path.join(dataset_dir, split, f"{split}_features.npy"), np.array(features))
    np.save(os.path.join(dataset_dir, split, f"{split}_labels.npy"), np.array(labels))
    print(f"✅ Saved EfficientNet {split} features and labels.")

In [5]:
# Run for all splits
efficientnet_dataset_dir = os.path.join(os.getcwd(), "efficientnet_dataset")
for split in ["train", "test", "val"]:
    extract_features_efficientnet(efficientnet_dataset_dir, split)

EfficientNet train: 100%|██████████| 21715/21715 [22:36<00:00, 16.01it/s]


✅ Saved EfficientNet train features and labels.


EfficientNet test: 100%|██████████| 6788/6788 [06:52<00:00, 16.45it/s]


✅ Saved EfficientNet test features and labels.


EfficientNet val: 100%|██████████| 5417/5417 [05:34<00:00, 16.20it/s]

✅ Saved EfficientNet val features and labels.
